# productpitchedをうまく予測すると効くかもね、という話
強化学習に近いのかも。というか、成約したいなら成約確率を出すより、「どのproductを推薦すべきか」を出すべきで、そうすると強化学習モデルを作ってあげたほうが顧客価値高そう

In [1]:
import unicodedata
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

##  各種定数

In [12]:
# 使用するカラムのリストを作成
package_column = "ProductPitched_str"
selected_columns = [
    "CityTier",
    "Occupation",
    "Gender(is_male)",
    "NumberOfPersonVisiting",
    "NumberOfFollowups",
    "PreferredPropertyStar",
    "NumberOfTrips",
    "Passport",
    "Designation",
    "Age",
    "MonthlyIncome_numeric",
    "car",
    "children",
    "TypeofContact_Company Invited",
    "TypeofContact_Self Enquiry",
    "TypeofContact_unknown",
    "marriage_history_未婚",
    "marriage_history_独身",
    "marriage_history_結婚済み",
    "marriage_history_離婚済み",
    "ProdTaken"
]
state_columns = selected_columns[:-1]

In [3]:
# 変な文字をアルファベットに
conv2alphabet_dict = {
    'α': 'a',
    'Α': 'a',
    'в': 'b',
    'β': 'b',
    '𐊡': 'b',
    'ς': 'c',
    'ϲ': 'c',
    'с': 'c',
    '𝔡': 'd',
    'ᗞ': 'd',
    'ꭰ': 'd',
    'ε': 'e',
    'ı': 'i',
    '|': 'l',
    'ո': 'n',
    'տ': 's',
    'ꓢ': 's',
    'ѕ': 's',
    '×': 'x'
}

def zenkaku2hankaku(text):
    return unicodedata.normalize('NFKC', text)


def conv2alphabet(text, replacements):
    t = str.maketrans(replacements)
    return text.translate(t)


def product_fix(
    df_preprocessed: pd.DataFrame,
    col_name: str = "ProductPitched_str"
) -> pd.DataFrame:
    df_preprocessed[col_name] = df_preprocessed[col_name].apply(
        lambda x: conv2alphabet(x, conv2alphabet_dict)
    )
    # 全角を半角に
    df_preprocessed[col_name] = df_preprocessed[col_name].apply(zenkaku2hankaku)
    # 大文字を小文字に
    df_preprocessed[col_name] = df_preprocessed[col_name].str.lower()
    # なぜか変換できないものたちをパワーで変換
    conv_dict = {
        'вasic': 'basic',
        'ѕuper deluxe': 'super deluxe',
        'baտic': 'basic',
        'ꭰeluxe': 'deluxe',
        'βasic': 'basic',
        'տuper deluxe': 'super deluxe',
        'տtandard': 'standard',
        'standarꭰ': 'standard',
        'basiс': 'basic',
        'dεluxε': 'deluxe',
        'basιc': 'basic',
        'super ꭰeluxe': 'super deluxe',
        'deluxε': 'deluxe',
        'ѕtandard': 'standard',
        'super dεluxe': 'super deluxe',
        'βasiс': 'basic',
        'supεr ꭰeluxe': 'super deluxe',
        'basιс': 'basic',
        'baѕic': 'basic'
    }
    df_preprocessed[col_name] = df_preprocessed[col_name].replace(conv_dict)
    
    return df_preprocessed


In [4]:
# データの読み込み
row_train_df = pd.read_csv("../data/train.csv")
train_file_path = "../data/20240812/train_preprocessed.csv"
train_df = pd.read_csv(train_file_path)

In [22]:
test_file_path = "../data/20240812/test_preprocessed.csv"
test_df = pd.read_csv(test_file_path)

In [5]:
# 必要なカラムのみを抽出
filtered_train_df = train_df[selected_columns]
filtered_train_df["ProductPitched_str"] = row_train_df["ProductPitched"].astype(str)
filtered_train_df = product_fix(filtered_train_df)
filtered_test_df = test_df[selected_columns[:-1]]

/var/folders/5_/6wgs064d6yj6tsvtkjt8g_740000gn/T/ipykernel_11211/2672106238.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["ProductPitched_str"] = row_train_df["ProductPitched"].astype(str)
/var/folders/5_/6wgs064d6yj6tsvtkjt8g_740000gn/T/ipykernel_11211/1000164297.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessed[col_name] = df_preprocessed[col_name].apply(
/var/folders/5_/6wgs064d6yj6tsvtkjt8g_740000gn/T/ipykernel_11211/1000164297.py:41: SettingWithCopyWarning: 
A

In [6]:
filtered_train_df.head()

,CityTier,Occupation,Gender(is_male),NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,Designation,Age,...,children,TypeofContact_Company Invited,TypeofContact_Self Enquiry,TypeofContact_unknown,marriage_history_未婚,marriage_history_独身,marriage_history_結婚済み,marriage_history_離婚済み,ProdTaken,ProductPitched_str
0,0.5,1.0,1.0,0.0,0.666667,0.0,0.625,1.0,0.0,0.531250,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,basic
1,0.0,0.0,1.0,0.0,0.666667,0.0,0.250,1.0,0.5,0.593750,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,standard
2,0.0,1.0,0.0,0.0,0.500000,0.0,0.500,0.0,0.0,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,basic
3,0.5,0.5,0.0,0.0,0.500000,0.5,0.125,0.0,0.5,0.770833,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,standard
4,1.0,0.5,0.0,0.0,0.500000,0.5,0.500,0.0,0.0,0.510417,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,basic


In [7]:
# パッケージを一意のIDにマッピング
ord_encoder = OrdinalEncoder(categories=[["basic", "standard", "deluxe", "super deluxe", "king"]])
filtered_train_df["ProductPitched"] = ord_encoder.fit_transform(filtered_train_df[[package_column]])

/var/folders/5_/6wgs064d6yj6tsvtkjt8g_740000gn/T/ipykernel_11211/3862666617.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["ProductPitched"] = ord_encoder.fit_transform(filtered_train_df[[package_column]])


In [8]:
filtered_train_df.head()

,CityTier,Occupation,Gender(is_male),NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,Designation,Age,...,TypeofContact_Company Invited,TypeofContact_Self Enquiry,TypeofContact_unknown,marriage_history_未婚,marriage_history_独身,marriage_history_結婚済み,marriage_history_離婚済み,ProdTaken,ProductPitched_str,ProductPitched
0,0.5,1.0,1.0,0.0,0.666667,0.0,0.625,1.0,0.0,0.531250,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,basic,0.0
1,0.0,0.0,1.0,0.0,0.666667,0.0,0.250,1.0,0.5,0.593750,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,standard,1.0
2,0.0,1.0,0.0,0.0,0.500000,0.0,0.500,0.0,0.0,0.000000,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,basic,0.0
3,0.5,0.5,0.0,0.0,0.500000,0.5,0.125,0.0,0.5,0.770833,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,standard,1.0
4,1.0,0.5,0.0,0.0,0.500000,0.5,0.500,0.0,0.0,0.510417,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,basic,0.0


In [10]:
filtered_train_df = filtered_train_df.drop(columns=[package_column], inplace=False)

In [26]:
# データの分割
train_df, test_df = train_test_split(filtered_train_df, test_size=0.2, random_state=42)

In [13]:
# Q-learningの基本的な設定
class QLearningAgent:
    def __init__(self, state_size, action_size, learning_rate=0.1, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01):
        """ Q-learningエージェントの初期化 """
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.q_table = defaultdict(lambda: np.zeros(action_size))
        
    def choose_action(self, state):
        """ 現在の状態に基づいて行動を選択 """
        if np.random.rand() < self.exploration_rate:
            return random.choice(range(self.action_size))
        return np.argmax(self.q_table[state])
    
    def learn(self, state, action, reward, next_state):
        """ Q値を更新する """
        best_next_action = np.argmax(self.q_table[next_state])
        td_target = reward + self.discount_factor * self.q_table[next_state][best_next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.learning_rate * td_error
        
        # 探索率の減少
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay

# Q-learningの行動としてパッケージIDを使用
action_size = 5  # パッケージの種類数

# エージェントの初期化
state_size = len(state_columns)
agent = QLearningAgent(state_size=state_size, action_size=action_size)

In [33]:
# エピソードの学習プロセス
num_episodes = 1000

for episode in range(num_episodes):
    for index, row in train_df.iterrows():
        state = tuple(row[state_columns])
        action = agent.choose_action(state)
        
        # 正しいパッケージと提案されたパッケージの差を計算
        true_package = row["ProductPitched"]
        distance = abs(true_package - action)
        
        # ProdTaken が 1 のとき：正しいパッケージを提案できた場合に報酬
        if row["ProdTaken"] == 1:
            reward = 100 if true_package == action else 0.0
        
        # ProdTaken が 0 のとき：距離が大きいほどペナルティを増加
        else:
            reward = -(distance)  # 距離が最大でペナルティが -5.0 になる

        next_state = state  # 次の状態が同じと仮定

        agent.learn(state, action, reward, next_state)

# 学習後のQテーブルを表示
print(agent.q_table)

defaultdict(<function QLearningAgent.__init__.<locals>.<lambda> at 0x157680af0>, {(0.0, 0.0, 1.0, 0.33333334, 1.0, 0.0, 0.5, 0.0, 0.0, 0.40625, 0.44665, 0.0, 0.33333334, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0): array([ 0.        , -0.38835078, -0.3842795 , -0.685425  , -0.19      ]), (0.5, 0.5, 1.0, 0.0, 0.5, 0.5, 0.25, 0.0, 0.0, 0.47916666, 0.1529875, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0): array([1986.35465299,  193.29636314,  360.66708049,  189.3089967 ,
        337.76263888]), (1.0, 0.0, 1.0, 0.33333334, 0.6666667, 0.0, 0.625, 0.0, 0.25, 0.6145833, 0.25, 1.0, 0.33333334, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0): array([-0.7501295, -0.21025  ,  0.       , -0.21025  , -0.720605 ]), (0.5, 1.0, 1.0, 0.0, 0.6666667, 0.5, 0.125, 0.0, 0.0, 0.39583334, 0.1598125, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0): array([1986.4907655 ,  189.03606516,  191.64547423,  475.31724892,
          4.89974294]), (0.5, 0.0, 0.0, 0.6666667, 0.6666667, 0.0, 0.625, 1.0, 0.75, 0.46875, 0.9383875, 1.0, 0.6666667, 0

In [34]:
test_df.head()

,CityTier,Occupation,Gender(is_male),NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,Designation,Age,...,TypeofContact_unknown,marriage_history_未婚,marriage_history_独身,marriage_history_結婚済み,marriage_history_離婚済み,ProdTaken,ProductPitched,PredictedPackage,PredictedPackage_str,ProductPitched_str
295,0.5,0.0,1.0,0.666667,0.666667,0.0,0.000,0.0,0.75,0.500000,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0,basic,super deluxe
2868,0.0,0.5,0.0,1.000000,0.000000,0.0,0.375,0.0,0.25,0.385417,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0,basic,deluxe
3374,0.0,0.0,1.0,1.000000,0.666667,0.5,0.250,0.0,0.50,0.562500,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,basic,standard
1462,0.5,0.5,1.0,0.666667,0.833333,0.0,0.875,0.0,0.00,0.364583,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,basic,basic
969,0.0,0.5,1.0,0.333333,0.500000,0.0,0.500,0.0,0.00,0.468750,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,basic,basic


In [35]:
# テストデータに対する予測
predictions = []
for index, row in test_df.iterrows():
    state = tuple(row[state_columns])
    predicted_action = agent.choose_action(state)
    predictions.append(predicted_action)

# 予測結果をテストデータに追加
test_df['PredictedPackage'] = predictions

# パッケージIDを元の名前に戻す（Optional）
test_df["ProductPitched_str"] = ord_encoder.inverse_transform(test_df[['ProductPitched']]).flatten()
test_df["PredictedPackage_str"] = ord_encoder.inverse_transform(test_df[['PredictedPackage']]).flatten()

# 結果の保存（必要に応じて）
test_df.to_csv("../data/20240815/test_predictions.csv", index=False)

# 予測結果の確認
print(test_df[['ProductPitched_str', 'PredictedPackage_str']].head())
test_df["PredictedPackage_str"].value_counts()

     ProductPitched_str PredictedPackage_str
295        super deluxe                basic
2868             deluxe                basic
3374           standard                basic
1462              basic                basic
969               basic                basic


PredictedPackage_str
basic           696
super deluxe      1
standard          1
Name: count, dtype: int64